In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
df = spark.read.csv(
    "S5PW.tsv", sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0008,you know when you have sort of a close up of their head
S0012,yeah
S0008,they be almost life size but not overpoweringly big you know
S0012,no
S0013,mm


In [3]:
def extract_speakers_n(df1, n):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0] not in speaker:
            speaker.append(row[0])
        if len(speaker) == n:
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [4]:
def extract_dialogues_dic_n(rdd1, speaker):
    n = len(speaker)
    dataC = rdd1.collect()
    speaker_dic = [{} for x in range(0,n)]
    
    
    for i in range(0,len(dataC)):
        if dataC[i][0] in speaker:
            p = speaker.index(dataC[i][0])
            if dataC[i][1]!=None:
                speaker_dic[p][i] = dataC[i][1].split()
    
    return speaker, speaker_dic

In [5]:
def get_er_one_with_u(x,y, exclude_stops):
    record=[[0 for i in range(len(y)+1)]  for j in range(len(x)+1)]
    dic_record_s1 = {}
    dic_record_s2 = {}
    expression_list_1 = []
    expression_list_2 = []
    expression_list_1_token = []
    expression_list_2_token = []
    for i in range(0,len(x)):
        for j in range(0,len(y)):
            if x[i] == y[j]:
                record[i+1][j+1] = record[i][j] + 1
                if record[i+1][j+1] > 0:
                        if i not in dic_record_s1.keys():
                            dic_record_s1[i] = record[i+1][j+1]
                        elif i in dic_record_s1.keys():
                            if (dic_record_s1[i]<record[i+1][j+1]):
                                dic_record_s1[i] = record[i+1][j+1]
                                
                        if j not in dic_record_s2.keys():
                            dic_record_s2[j] = record[i+1][j+1]
                        elif j in dic_record_s2.keys():
                            if (dic_record_s2[j]<record[i+1][j+1]):
                                dic_record_s2[j] = record[i+1][j+1]

    if dic_record_s1== {}:
        return []
    
    else:
        i = max(dic_record_s1.values())
        
        
    if dic_record_s2== {}:
        return []
    
    else:
        j = max(dic_record_s2.values())
    
    while i>0:  
        for key in dic_record_s1:
            if dic_record_s1[key] == i:
            #expression_repitition_record = expression_repitition_record+1
                expression = x[key+1-i : key+1]
                expression_string = ' '.join(expression)
            
                for k in range(0,i):
                    dic_record_s1[key - k] = 0
            
                if exclude_stops == True:
                    if expression_string not in stops:
                        if expression_string not in expression_list_1:
                            expression_list_1.append(expression_string)
                            expression_list_1_token.append(expression)
                                     
                else:
                    if expression_string not in expression_list_1:
                        expression_list_1.append(expression_string)
                        expression_list_1_token.append(expression)
        
        i = i-1
        
    while j>0:  
        for key in dic_record_s2:
            if dic_record_s2[key] == j:
                expression_2 = y[key+1-j : key+1]
                expression_string_2 = ' '.join(expression_2)
            
                for k in range(0,j):
                    dic_record_s2[key - k] = 0
                
                
                
                if exclude_stops == True:
                    if expression_string_2 not in stops:
                        if expression_string_2 not in expression_list_1:
                            expression_list_1.append(expression_string_2)
                            expression_list_1_token.append(expression_2)
                
                else:
                    if expression_string_2 not in expression_list_1:
                        expression_list_1.append(expression_string_2)
                        expression_list_1_token.append(expression_2)
        
        j = j-1
        
    return expression_list_1

In [6]:
def get_er_dic_in_window(speaker1_dialogue, speaker2_dialogue, u_low, u_high):
    expressions = {}
    
    for x,value_x in speaker1_dialogue.items():
        for y,value_y in speaker2_dialogue.items():
            if (x<u_high) and (x>= u_low) and (y<u_high) and (y>= u_low):
                string = get_er_one_with_u(value_x,value_y, False)
            
                for s in string:
                
                    if s not in expressions.keys():
                    
                        pos = max(x,y)
                        expressions[s] = [pos,[x],[y]]
                    
                    elif s in expressions.keys():
                        if (x in expressions[s][1]) and (y not in expressions[s][2]):
                            expressions[s][2].append(y)
                            expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
                        
                        elif (y in expressions[s][2]) and (x not in expressions[s][1]):
                            expressions[s][1].append(x)
                            expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
    
    return expressions

In [7]:
def get_er_dic_group_in_window(speaker, speaker_dic,u_low, u_high):
    n = len(speaker)
    expression_n = []
    fit_ex_in_dic = [[]*n]
    sum_dic = {}
    
    for i in range(0,n-1):
        for j in range(i+1, n):
            ex = get_er_dic_in_window(speaker_dic[i],speaker_dic[j],u_low, u_high)
            expression_n.append(ex)
            
    for split_dic in expression_n:
        for k, v in split_dic.items():
            v_sum = []
            pos = v[0]
            
            for v1 in v[1]:
                v_sum.append(v1)
            for v2 in v[2]:
                v_sum.append(v2)
            
            if k not in sum_dic.keys():
                sum_dic[k] = [pos,v_sum]
            else:
                for vs in v_sum:
                    if vs not in sum_dic[k][1]:
                        sum_dic[k][1].append(vs)
                        sum_dic[k][0] = min(pos, sum_dic[k][0])
    
    
    for key, value in sum_dic.items():
        sum_dic[key][1] = sorted(sum_dic[key][1])
        
    return sum_dic
        

In [8]:
def get_er_ratio_in_time_n(speaker, speaker_dic, u_low, u_high):
    ex_in_window = get_er_dic_group_in_window(speaker, speaker_dic,u_low, u_high)
#     print(ex_in_window.keys())
    n = len(speaker)
    sum_token = [0 for i in range(0,n)]
    max_len = 0
    speaker_dic_string = [{} for i in range(0,n)]
    sum_er = [0 for i in range(0,n)]
    sum_er_token = [0 for i in range(0,n)]
    er_ratio = [0 for i in range(0,n)]
#     print(speaker_dic[2])
    
    for key,value in ex_in_window.items():
        max_len = max(max_len, len(key.split()))
    
    for i in range(0,n):
        for k,v in speaker_dic[i].items():
            if k<u_high and k>=u_low:
                sum_token[i] = sum_token[i]+len(speaker_dic[i][k])
                speaker_dic_string[i][k] = ' '.join(speaker_dic[i][k])
    
    m = max_len
    
    while m>1:
        for k,v in ex_in_window.items():
            if (len(k.split())==m):
                for u in ex_in_window[k][1]:
                    if u<u_high and u>= u_low:
                        for j in range(0,len(speaker_dic_string)):
                            if u in speaker_dic_string[j].keys():
                                c = speaker_dic_string[j][u].count(k)
                                speaker_dic_string[j][u] = speaker_dic_string[j][u].replace(k, '0'*len(k))
                                if u>= ex_in_window[k][0]:
                                    sum_er[j] = sum_er[j]+ c
                                    sum_er_token[j] = sum_er_token[j] + c*len(k.split())
        m = m-1
        
    for k,v in ex_in_window.items():
        if (len(k.split())==1):
            for u in ex_in_window[k][1]:
                if u<u_high and u>= u_low:
                    for j in range(0,len(speaker_dic_string)):
                        if u in speaker_dic_string[j].keys():
                            c = speaker_dic_string[j][u].split().count(k)
                            if u>= ex_in_window[k][0]:
                                sum_er[j] = sum_er[j]+ c
                                sum_er_token[j] = sum_er_token[j]+c
                            
    for c in range(0,len(sum_token)):
        if sum_token[c]!=0:
            er_ratio[c] = sum_er_token[c]/sum_token[c]
        else:
            er_ratio[c] = 0
    
    total_token_number = 0
    for tn in sum_token:
        total_token_number = total_token_number + tn
    
    total_er_token_number = 0
    for en in sum_er_token:
        total_er_token_number = total_er_token_number + en
        
    total_er_number = 0
    for er_n in sum_er:
        total_er_number = total_er_number + er_n
    
    if total_token_number!=0:
        av_er_ratio = total_er_token_number/total_token_number
    else:
        av_er_ratio = 0
    
    if n==3:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
    if n==4:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'speaker4': speaker[3], 'speaker4_token_number': sum_token[3], 'speaker4_expression_token_number':sum_er_token[3], 
                      'speaker4_expression_number':sum_er[3],'speaker4_er_ratio':er_ratio[3],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
        
    
    return result_dic
    
        
        

In [9]:
def number_of_slot_n(speaker_dic,length_u):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    number_slot = int(number_of_u - number_of_u%length_u)/length_u
    return int(number_slot)

In [10]:
def number_of_total_u(speaker_dic):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    return number_of_u

In [11]:
def get_er_ratio_temporal_n(speaker, speaker_dic, length):
    slot_number = number_of_slot_n(speaker_dic, length)
    er_dic_list = []
    total_u = 0
    
    for s in speaker_dic:
        total_u = total_u+len(s)
    
    for i in range(0,slot_number):
        er_dic = get_er_ratio_in_time_n(speaker,speaker_dic, length*i, length*(i+1))
        er_dic['begin'] = length*i
        er_dic['end'] = length*(i+1)
        er_dic_list.append(er_dic)
    
    er_dic_end = get_er_ratio_in_time_n(speaker, speaker_dic, slot_number*length, total_u)
    er_dic_end['begin'] = slot_number*length
    er_dic_end['end'] = total_u
    er_dic_list.append(er_dic_end)
    
    return er_dic_list

In [12]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(len(stops))
filled_p = ['ah','oh','eh','er','erm','mm','mhm','huh','uhu']
for p in filled_p:
    stops.add(p)
print(stops)
print(len(stops))

179
{'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', "hasn't", 'huh', 'then', 'both', 'so', 're', 'don', 'm', 'as', 'any', 'mightn', 'after', 'you', 'wouldn', 'why', 'been', 'where', 'by', "isn't", 'yourself', 'wasn', 'a', 'er', "haven't", 'did', "hadn't", 'uhu', 'their', 'hasn', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', "you'll", 'yourselves', 'that', 'my', 'what', 'to', 'd', 'not', "won't", "couldn't", 'own', 'there', 'this', 'mhm', 'each', 'all', 'haven', 'more', 'me', 've', 'weren', 'which', 'himself', 'nor', 'other', "shouldn't", 'who', "should've", 'erm', 'same', 'at', 'such', 't', 'up', 'than', 'mm', 'can', "you've", 'too', 'ah', 'these', 'while', "wasn't", 'ourselves', 'before', 'i', 'he', "didn't", 'our', 'its', 'but', 'with', "wouldn't", 'those', 'because', 'the', 'y', 'shouldn', 'it', 'mustn', 'hers', 'just', 'doesn', 'ain', 'between', 'over', 'had', 'aren', "mightn't", 'does', 'have', 'and', 'or', 'some', "mustn't", 'only', 'won', 'when', 'needn', '

In [13]:
out1 = extract_speakers_n(df,4)
out2 = extract_dialogues_dic_n(out1[0],out1[1])
out3 = get_er_dic_group_in_window(out2[0], out2[1], 0, 50)

In [14]:
print(out2[0])

['S0008', 'S0012', 'S0013', 'S0024']


In [15]:
for k,v in out3.items():
    print(k,',',v)

you , [25, [0, 2, 7, 9, 11, 16, 25, 30, 31, 38, 39, 45, 47]]
have , [25, [0, 9, 24, 25, 38, 41]]
a , [25, [0, 9, 16, 25, 33, 48]]
when , [33, [0, 33]]
close , [39, [0, 21, 39]]
up , [39, [0, 39]]
be , [15, [2, 5, 9, 15, 24, 41, 47, 49]]
they , [35, [2, 35, 37]]
big , [35, [2, 11, 19, 35, 46]]
the , [39, [5, 9, 11, 21, 39, 41, 43]]
all , [48, [5, 48]]
you get , [25, [7, 25]]
so , [25, [7, 9, 21, 25, 31, 38, 39]]
get , [25, [7, 9, 21, 25, 41, 42, 46]]
so you , [31, [7, 9, 31, 38, 39]]
don't , [37, [7, 37]]
that , [27, [9, 14, 27, 33, 41, 47, 49]]
out , [33, [9, 33]]
and , [35, [9, 21, 24, 35, 45, 46]]
i , [10, [9, 10, 39, 41, 43, 47]]
four , [39, [9, 39]]
we , [48, [9, 48]]
it , [25, [14, 24, 25, 31, 38, 39, 41, 43, 45]]
it's , [33, [14, 33, 39]]
something , [33, [16, 33]]
well , [33, [21, 33, 41]]
to , [39, [21, 24, 38, 39, 43, 45]]
yeah , [26, [1, 8, 12, 18, 23, 26, 28, 29, 33, 34, 36, 42]]
you can , [31, [30, 31]]
that's right , [32, [6, 32, 36]]
auto , [32, [31, 32, 33]]
three , [39,

In [16]:
out4 = get_er_ratio_in_time_n(out2[0], out2[1], 0, 50)

In [17]:
print(out4)

{'speaker1': 'S0008', 'speaker1_token_number': 209, 'speaker1_expression_token_number': 44, 'speaker1_expression_number': 37, 'speaker1_er_ratio': 0.21052631578947367, 'speaker2': 'S0012', 'speaker2_token_number': 90, 'speaker2_expression_token_number': 50, 'speaker2_expression_number': 44, 'speaker2_er_ratio': 0.5555555555555556, 'speaker3': 'S0013', 'speaker3_token_number': 16, 'speaker3_expression_token_number': 7, 'speaker3_expression_number': 6, 'speaker3_er_ratio': 0.4375, 'speaker4': 'S0024', 'speaker4_token_number': 0, 'speaker4_expression_token_number': 0, 'speaker4_expression_number': 0, 'speaker4_er_ratio': 0, 'total_token_number': 315, 'total_er_token_number': 101, 'av_er_ratio': 0.32063492063492066, 'total_er_number': 87}


In [18]:
out5 = number_of_slot_n(out2[1], 50)

In [19]:
out6 = get_er_ratio_temporal_n(out2[0], out2[1], 50)

In [20]:
for o in out6:
    print(o)

{'speaker1': 'S0008', 'speaker1_token_number': 209, 'speaker1_expression_token_number': 44, 'speaker1_expression_number': 37, 'speaker1_er_ratio': 0.21052631578947367, 'speaker2': 'S0012', 'speaker2_token_number': 90, 'speaker2_expression_token_number': 50, 'speaker2_expression_number': 44, 'speaker2_er_ratio': 0.5555555555555556, 'speaker3': 'S0013', 'speaker3_token_number': 16, 'speaker3_expression_token_number': 7, 'speaker3_expression_number': 6, 'speaker3_er_ratio': 0.4375, 'speaker4': 'S0024', 'speaker4_token_number': 0, 'speaker4_expression_token_number': 0, 'speaker4_expression_number': 0, 'speaker4_er_ratio': 0, 'total_token_number': 315, 'total_er_token_number': 101, 'av_er_ratio': 0.32063492063492066, 'total_er_number': 87, 'begin': 0, 'end': 50}
{'speaker1': 'S0008', 'speaker1_token_number': 318, 'speaker1_expression_token_number': 130, 'speaker1_expression_number': 122, 'speaker1_er_ratio': 0.4088050314465409, 'speaker2': 'S0012', 'speaker2_token_number': 172, 'speaker2_ex

In [21]:
for o in out6:
    print(o['av_er_ratio'])

0.32063492063492066
0.4277227722772277
0.39119804400978
0.35714285714285715
0.4192546583850932
0.3938223938223938
0.2765273311897106
0.3673469387755102
0.34068627450980393
0.4365781710914454
0.2966101694915254
0.3136094674556213
0.36824324324324326
0.3877005347593583
0.3310344827586207
0.3333333333333333
0.38
0.36728395061728397
0.3265895953757225
0.4036363636363636
0.46511627906976744
0.42323651452282157
0.3888888888888889
0.426
0.4371584699453552
0.42443729903536975
0.5140997830802603
0.4243243243243243
0.45308924485125857
0.43661971830985913
0.3291139240506329
0.31024930747922436
0.4460431654676259
0.185378590078329
0.45498783454987834
0.38837920489296635
0.4416961130742049
0.38328530259365995
0.44715447154471544
0.29210526315789476
0.4028268551236749
0.3956386292834891
0.48120300751879697
0.30823529411764705
0.2911392405063291
0.1983695652173913
0.41379310344827586
0.32142857142857145
0.0


In [23]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-4/output_er_summary_4_1_dialign_in_window'
dl = []
dl_result = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,4)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
#     ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
    
    er_result_f = get_er_ratio_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1], 50)
    for fx in er_result_f:
        fx['file'] = f
        
#     er_result_f_2 = get_er_ratio_in_time_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 0, number_of_total_u(dialogue_dic_f[1]))
#     er_result_f_2['file'] = f
#     print(er_result_f_2)
#     dl_result.append(er_result_f_2)
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)

# dl_r = pd.DataFrame(dl_result)
# dl_r.to_csv(os.path.join(path,'general_result1_new.csv'), index = False)

In [44]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-3/output_er_summary_3_dialign'
dl = []
dl_result = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,3)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
    
    er_result_f = get_er_ratio_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 50)
    for fx in er_result_f:
        fx['file'] = f
        
    er_result_f_2 = get_er_ratio_in_time_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 0, number_of_total_u(dialogue_dic_f[1]))
    er_result_f_2['file'] = f
    print(er_result_f_2)
    dl_result.append(er_result_f_2)
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)

dl_r = pd.DataFrame(dl_result)
dl_r.to_csv(os.path.join(path,'general_result1_new.csv'), index = False)

{'speaker1': 'S0420', 'speaker1_token_number': 1298, 'speaker1_expression_token_number': 867, 'speaker1_expression_number': 707, 'speaker1_er_ratio': 0.6679506933744221, 'speaker2': 'S0366', 'speaker2_token_number': 575, 'speaker2_expression_token_number': 261, 'speaker2_expression_number': 213, 'speaker2_er_ratio': 0.4539130434782609, 'speaker3': 'S0365', 'speaker3_token_number': 465, 'speaker3_expression_token_number': 262, 'speaker3_expression_number': 195, 'speaker3_er_ratio': 0.5634408602150538, 'total_token_number': 2338, 'total_er_token_number': 1390, 'av_er_ratio': 0.5945252352437981, 'total_er_number': 1115, 'file': 'SWHR.tsv'}
{'speaker1': 'S0047', 'speaker1_token_number': 8620, 'speaker1_expression_token_number': 5529, 'speaker1_expression_number': 4277, 'speaker1_er_ratio': 0.641415313225058, 'speaker2': 'S0018', 'speaker2_token_number': 3511, 'speaker2_expression_token_number': 2382, 'speaker2_expression_number': 1715, 'speaker2_er_ratio': 0.6784391911136428, 'speaker3': '

{'speaker1': 'S0653', 'speaker1_token_number': 5691, 'speaker1_expression_token_number': 3942, 'speaker1_expression_number': 2888, 'speaker1_er_ratio': 0.6926726410121244, 'speaker2': 'S0654', 'speaker2_token_number': 2777, 'speaker2_expression_token_number': 1774, 'speaker2_expression_number': 1256, 'speaker2_er_ratio': 0.6388188692833994, 'speaker3': 'S0655', 'speaker3_token_number': 2058, 'speaker3_expression_token_number': 1308, 'speaker3_expression_number': 928, 'speaker3_er_ratio': 0.6355685131195336, 'total_token_number': 10526, 'total_er_token_number': 7024, 'av_er_ratio': 0.66730001900057, 'total_er_number': 5072, 'file': 'S8LG.tsv'}
{'speaker1': 'S0013', 'speaker1_token_number': 1814, 'speaker1_expression_token_number': 1150, 'speaker1_expression_number': 878, 'speaker1_er_ratio': 0.6339581036383682, 'speaker2': 'S0012', 'speaker2_token_number': 1268, 'speaker2_expression_token_number': 915, 'speaker2_expression_number': 677, 'speaker2_er_ratio': 0.7216088328075709, 'speaker3

{'speaker1': 'S0013', 'speaker1_token_number': 2860, 'speaker1_expression_token_number': 2028, 'speaker1_expression_number': 1493, 'speaker1_er_ratio': 0.7090909090909091, 'speaker2': 'S0370', 'speaker2_token_number': 5251, 'speaker2_expression_token_number': 3183, 'speaker2_expression_number': 2390, 'speaker2_er_ratio': 0.6061702532850886, 'speaker3': 'S0012', 'speaker3_token_number': 4135, 'speaker3_expression_token_number': 2941, 'speaker3_expression_number': 2064, 'speaker3_er_ratio': 0.7112454655380894, 'total_token_number': 12246, 'total_er_token_number': 8152, 'av_er_ratio': 0.6656867548587294, 'total_er_number': 5947, 'file': 'SJH6.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 1430, 'speaker1_expression_token_number': 795, 'speaker1_expression_number': 652, 'speaker1_er_ratio': 0.5559440559440559, 'speaker2': 'S0655', 'speaker2_token_number': 830, 'speaker2_expression_token_number': 468, 'speaker2_expression_number': 341, 'speaker2_er_ratio': 0.563855421686747, 'speaker3

{'speaker1': 'S0454', 'speaker1_token_number': 7493, 'speaker1_expression_token_number': 5567, 'speaker1_expression_number': 4075, 'speaker1_er_ratio': 0.7429600960896837, 'speaker2': 'S0579', 'speaker2_token_number': 5322, 'speaker2_expression_token_number': 3506, 'speaker2_expression_number': 2596, 'speaker2_er_ratio': 0.6587748966553927, 'speaker3': 'S0580', 'speaker3_token_number': 3494, 'speaker3_expression_token_number': 2211, 'speaker3_expression_number': 1605, 'speaker3_er_ratio': 0.6327990841442472, 'total_token_number': 16309, 'total_er_token_number': 11284, 'av_er_ratio': 0.6918879146483536, 'total_er_number': 8276, 'file': 'SPYH.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 1246, 'speaker1_expression_token_number': 705, 'speaker1_expression_number': 569, 'speaker1_er_ratio': 0.5658105939004815, 'speaker2': 'S0655', 'speaker2_token_number': 563, 'speaker2_expression_token_number': 350, 'speaker2_expression_number': 288, 'speaker2_er_ratio': 0.6216696269982238, 'speake

{'speaker1': 'S0266', 'speaker1_token_number': 1002, 'speaker1_expression_token_number': 664, 'speaker1_expression_number': 506, 'speaker1_er_ratio': 0.6626746506986028, 'speaker2': 'S0310', 'speaker2_token_number': 4626, 'speaker2_expression_token_number': 2439, 'speaker2_expression_number': 1967, 'speaker2_er_ratio': 0.5272373540856031, 'speaker3': 'S0267', 'speaker3_token_number': 620, 'speaker3_expression_token_number': 461, 'speaker3_expression_number': 341, 'speaker3_er_ratio': 0.7435483870967742, 'total_token_number': 6248, 'total_er_token_number': 3564, 'av_er_ratio': 0.5704225352112676, 'total_er_number': 2814, 'file': 'SVLK.tsv'}
{'speaker1': 'S0486', 'speaker1_token_number': 3287, 'speaker1_expression_token_number': 2288, 'speaker1_expression_number': 1646, 'speaker1_er_ratio': 0.6960754487374505, 'speaker2': 'S0489', 'speaker2_token_number': 2327, 'speaker2_expression_token_number': 1483, 'speaker2_expression_number': 1067, 'speaker2_er_ratio': 0.6373012462397937, 'speaker3

...

{'speaker1': 'S0152', 'speaker1_token_number': 3923, 'speaker1_expression_token_number': 2075, 'speaker1_expression_number': 1587, 'speaker1_er_ratio': 0.5289319398419576, 'speaker2': 'S0012', 'speaker2_token_number': 3166, 'speaker2_expression_token_number': 2342, 'speaker2_expression_number': 1661, 'speaker2_er_ratio': 0.7397346809854706, 'speaker3': 'S0013', 'speaker3_token_number': 725, 'speaker3_expression_token_number': 464, 'speaker3_expression_number': 361, 'speaker3_er_ratio': 0.64, 'total_token_number': 7814, 'total_er_token_number': 4881, 'av_er_ratio': 0.6246480675710263, 'total_er_number': 3609, 'file': 'SYHW.tsv'}
{'speaker1': 'S0018', 'speaker1_token_number': 4766, 'speaker1_expression_token_number': 2843, 'speaker1_expression_number': 2147, 'speaker1_er_ratio': 0.5965169953839697, 'speaker2': 'S0014', 'speaker2_token_number': 5025, 'speaker2_expression_token_number': 3459, 'speaker2_expression_number': 2543, 'speaker2_er_ratio': 0.6883582089552239, 'speaker3': 'S0004', 

{'speaker1': 'S0227', 'speaker1_token_number': 9720, 'speaker1_expression_token_number': 7452, 'speaker1_expression_number': 5168, 'speaker1_er_ratio': 0.7666666666666667, 'speaker2': 'S0190', 'speaker2_token_number': 3703, 'speaker2_expression_token_number': 2269, 'speaker2_expression_number': 1532, 'speaker2_er_ratio': 0.6127464218201458, 'speaker3': 'S0192', 'speaker3_token_number': 10770, 'speaker3_expression_token_number': 6824, 'speaker3_expression_number': 4953, 'speaker3_er_ratio': 0.6336118848653668, 'total_token_number': 24193, 'total_er_token_number': 16545, 'av_er_ratio': 0.6838755011780268, 'total_er_number': 11653, 'file': 'S3JF.tsv'}
{'speaker1': 'S0567', 'speaker1_token_number': 1105, 'speaker1_expression_token_number': 568, 'speaker1_expression_number': 484, 'speaker1_er_ratio': 0.5140271493212669, 'speaker2': 'S0621', 'speaker2_token_number': 528, 'speaker2_expression_token_number': 367, 'speaker2_expression_number': 258, 'speaker2_er_ratio': 0.6950757575757576, 'spea

{'speaker1': 'S0262', 'speaker1_token_number': 3654, 'speaker1_expression_token_number': 2654, 'speaker1_expression_number': 1910, 'speaker1_er_ratio': 0.7263273125342091, 'speaker2': 'S0301', 'speaker2_token_number': 1714, 'speaker2_expression_token_number': 903, 'speaker2_expression_number': 686, 'speaker2_er_ratio': 0.5268378063010501, 'speaker3': 'S0304', 'speaker3_token_number': 4089, 'speaker3_expression_token_number': 2588, 'speaker3_expression_number': 1919, 'speaker3_er_ratio': 0.6329175837613108, 'total_token_number': 9457, 'total_er_token_number': 6145, 'av_er_ratio': 0.6497832293539177, 'total_er_number': 4515, 'file': 'SA6K.tsv'}
{'speaker1': 'S0486', 'speaker1_token_number': 5575, 'speaker1_expression_token_number': 3164, 'speaker1_expression_number': 2388, 'speaker1_er_ratio': 0.5675336322869955, 'speaker2': 'S0523', 'speaker2_token_number': 2285, 'speaker2_expression_token_number': 1519, 'speaker2_expression_number': 1070, 'speaker2_er_ratio': 0.6647702407002188, 'speak

{'speaker1': 'S0475', 'speaker1_token_number': 882, 'speaker1_expression_token_number': 226, 'speaker1_expression_number': 193, 'speaker1_er_ratio': 0.2562358276643991, 'speaker2': 'S0418', 'speaker2_token_number': 20, 'speaker2_expression_token_number': 6, 'speaker2_expression_number': 6, 'speaker2_er_ratio': 0.3, 'speaker3': 'S0417', 'speaker3_token_number': 782, 'speaker3_expression_token_number': 573, 'speaker3_expression_number': 490, 'speaker3_er_ratio': 0.7327365728900256, 'total_token_number': 1684, 'total_er_token_number': 805, 'av_er_ratio': 0.47802850356294535, 'total_er_number': 689, 'file': 'SKY2.tsv'}
{'speaker1': 'S0278', 'speaker1_token_number': 5207, 'speaker1_expression_token_number': 3117, 'speaker1_expression_number': 2414, 'speaker1_er_ratio': 0.5986172460149798, 'speaker2': 'S0013', 'speaker2_token_number': 2292, 'speaker2_expression_token_number': 1577, 'speaker2_expression_number': 1136, 'speaker2_er_ratio': 0.6880453752181501, 'speaker3': 'S0012', 'speaker3_tok

{'speaker1': 'S0653', 'speaker1_token_number': 3436, 'speaker1_expression_token_number': 2219, 'speaker1_expression_number': 1773, 'speaker1_er_ratio': 0.6458090803259604, 'speaker2': 'S0655', 'speaker2_token_number': 1180, 'speaker2_expression_token_number': 640, 'speaker2_expression_number': 468, 'speaker2_er_ratio': 0.5423728813559322, 'speaker3': 'S0654', 'speaker3_token_number': 1020, 'speaker3_expression_token_number': 680, 'speaker3_expression_number': 486, 'speaker3_er_ratio': 0.6666666666666666, 'total_token_number': 5636, 'total_er_token_number': 3539, 'av_er_ratio': 0.6279276082327893, 'total_er_number': 2727, 'file': 'SXC6.tsv'}
{'speaker1': 'S0255', 'speaker1_token_number': 14154, 'speaker1_expression_token_number': 9968, 'speaker1_expression_number': 6865, 'speaker1_er_ratio': 0.7042532146389713, 'speaker2': 'S0257', 'speaker2_token_number': 17543, 'speaker2_expression_token_number': 12542, 'speaker2_expression_number': 8588, 'speaker2_er_ratio': 0.7149290315225446, 'spea

In [25]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-3/output_er_summary_3_dialign_ex_sw'
path2 = '~/corpus/bnc_untagged_dialign/tsvfiles-3/output_er_summary_3_dialign_lexicon_ex_sw'

dl = []
dl_result = []
ex_general_summary = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,3)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
    
    er_result_f = get_er_ratio_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 50)
    for fx in er_result_f:
        fx['file'] = f
        
    er_result_f_2 = get_er_ratio_in_time_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 0, number_of_total_u(dialogue_dic_f[1]))
    er_result_f_2['file'] = f
    print(er_result_f_2)
    dl_result.append(er_result_f_2)
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)
    
    ex_for_each_file = []
    for ex, value in ex_dic_f.items():
        ex_for_each_file.append({'Expression':ex, 'Expression_length': len(ex.split()), 'Expression_established_position':value[0], 'Expression_uttered_position_speaker':value[1],'file':f})
    
    ex_general_summary.append({'Expression Lexicon Size without stopwords': len(ex_dic_f),'file':f})
    print({'Expression Lexicon Size without stopwords': len(ex_dic_f),'file':f})
    dl_ex_for_each_file = pd.DataFrame(ex_for_each_file)
    dl_ex_for_each_file.to_csv(os.path.join(path2,f[0:5] + 'csv'), index = False)
    
    

dl_r = pd.DataFrame(dl_result)
dl_r.to_csv(os.path.join(path,'general_result1_new.csv'), index = False)
dl_ex_general_summary = pd.DataFrame(ex_general_summary)
dl_ex_general_summary.to_csv(os.path.join(path2,'expression_lexicon_size_3_1_ex_sw.csv'), index = False)

{'speaker1': 'S0420', 'speaker1_token_number': 1298, 'speaker1_expression_token_number': 480, 'speaker1_expression_number': 320, 'speaker1_er_ratio': 0.3697996918335901, 'speaker2': 'S0366', 'speaker2_token_number': 575, 'speaker2_expression_token_number': 155, 'speaker2_expression_number': 107, 'speaker2_er_ratio': 0.26956521739130435, 'speaker3': 'S0365', 'speaker3_token_number': 465, 'speaker3_expression_token_number': 164, 'speaker3_expression_number': 97, 'speaker3_er_ratio': 0.35268817204301073, 'total_token_number': 2338, 'total_er_token_number': 799, 'av_er_ratio': 0.34174508126603936, 'total_er_number': 524, 'file': 'SWHR.tsv'}
{'Expression Lexicon Size without stopwords': 253, 'file': 'SWHR.tsv'}
{'speaker1': 'S0047', 'speaker1_token_number': 8620, 'speaker1_expression_token_number': 3695, 'speaker1_expression_number': 2443, 'speaker1_er_ratio': 0.4286542923433875, 'speaker2': 'S0018', 'speaker2_token_number': 3511, 'speaker2_expression_token_number': 1752, 'speaker2_expressi

{'speaker1': 'S0266', 'speaker1_token_number': 207, 'speaker1_expression_token_number': 102, 'speaker1_expression_number': 51, 'speaker1_er_ratio': 0.4927536231884058, 'speaker2': 'S0314', 'speaker2_token_number': 4066, 'speaker2_expression_token_number': 1418, 'speaker2_expression_number': 803, 'speaker2_er_ratio': 0.3487456960157403, 'speaker3': 'S0269', 'speaker3_token_number': 2962, 'speaker3_expression_token_number': 1293, 'speaker3_expression_number': 717, 'speaker3_er_ratio': 0.4365293720459149, 'total_token_number': 7235, 'total_er_token_number': 2813, 'av_er_ratio': 0.3888044229440221, 'total_er_number': 1571, 'file': 'S8NX.tsv'}
{'Expression Lexicon Size without stopwords': 687, 'file': 'S8NX.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 5691, 'speaker1_expression_token_number': 2758, 'speaker1_expression_number': 1704, 'speaker1_er_ratio': 0.48462484624846247, 'speaker2': 'S0654', 'speaker2_token_number': 2777, 'speaker2_expression_token_number': 1328, 'speaker2_expre

{'speaker1': 'S0259', 'speaker1_token_number': 1855, 'speaker1_expression_token_number': 681, 'speaker1_expression_number': 442, 'speaker1_er_ratio': 0.36711590296495955, 'speaker2': 'S0255', 'speaker2_token_number': 2527, 'speaker2_expression_token_number': 1049, 'speaker2_expression_number': 691, 'speaker2_er_ratio': 0.4151167392164622, 'speaker3': 'S0260', 'speaker3_token_number': 6, 'speaker3_expression_token_number': 2, 'speaker3_expression_number': 1, 'speaker3_er_ratio': 0.3333333333333333, 'total_token_number': 4388, 'total_er_token_number': 1732, 'av_er_ratio': 0.39471285323609845, 'total_er_number': 1134, 'file': 'S2NQ.tsv'}
{'Expression Lexicon Size without stopwords': 407, 'file': 'S2NQ.tsv'}
{'speaker1': 'S0392', 'speaker1_token_number': 1509, 'speaker1_expression_token_number': 495, 'speaker1_expression_number': 333, 'speaker1_er_ratio': 0.32803180914512925, 'speaker2': 'S0390', 'speaker2_token_number': 1298, 'speaker2_expression_token_number': 567, 'speaker2_expression_n

{'speaker1': 'S0525', 'speaker1_token_number': 6836, 'speaker1_expression_token_number': 3087, 'speaker1_expression_number': 1898, 'speaker1_er_ratio': 0.4515798712697484, 'speaker2': 'S0617', 'speaker2_token_number': 3618, 'speaker2_expression_token_number': 1905, 'speaker2_expression_number': 1167, 'speaker2_er_ratio': 0.5265339966832504, 'speaker3': 'S0614', 'speaker3_token_number': 1625, 'speaker3_expression_token_number': 785, 'speaker3_expression_number': 460, 'speaker3_er_ratio': 0.48307692307692307, 'total_token_number': 12079, 'total_er_token_number': 5777, 'av_er_ratio': 0.47826806854872095, 'total_er_number': 3525, 'file': 'S457.tsv'}
{'Expression Lexicon Size without stopwords': 1482, 'file': 'S457.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 1572, 'speaker1_expression_token_number': 498, 'speaker1_expression_number': 329, 'speaker1_er_ratio': 0.31679389312977096, 'speaker2': 'S0654', 'speaker2_token_number': 1001, 'speaker2_expression_token_number': 392, 'speaker2_

{'speaker1': 'S0589', 'speaker1_token_number': 1630, 'speaker1_expression_token_number': 857, 'speaker1_expression_number': 553, 'speaker1_er_ratio': 0.5257668711656441, 'speaker2': 'S0588', 'speaker2_token_number': 6007, 'speaker2_expression_token_number': 2875, 'speaker2_expression_number': 1839, 'speaker2_er_ratio': 0.4786082903279507, 'speaker3': 'S0590', 'speaker3_token_number': 6694, 'speaker3_expression_token_number': 2917, 'speaker3_expression_number': 1915, 'speaker3_er_ratio': 0.43576337018225275, 'total_token_number': 14331, 'total_er_token_number': 6649, 'av_er_ratio': 0.46395924918009906, 'total_er_number': 4307, 'file': 'SRL9.tsv'}
{'Expression Lexicon Size without stopwords': 1575, 'file': 'SRL9.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 3691, 'speaker1_expression_token_number': 1496, 'speaker1_expression_number': 932, 'speaker1_er_ratio': 0.4053102140341371, 'speaker2': 'S0655', 'speaker2_token_number': 1100, 'speaker2_expression_token_number': 592, 'speaker2_

...

{'speaker1': 'S0623', 'speaker1_token_number': 4353, 'speaker1_expression_token_number': 2359, 'speaker1_expression_number': 1456, 'speaker1_er_ratio': 0.541925109120147, 'speaker2': 'S0525', 'speaker2_token_number': 5946, 'speaker2_expression_token_number': 2712, 'speaker2_expression_number': 1715, 'speaker2_er_ratio': 0.45610494450050454, 'speaker3': 'S0622', 'speaker3_token_number': 1731, 'speaker3_expression_token_number': 814, 'speaker3_expression_number': 514, 'speaker3_er_ratio': 0.4702484113229347, 'total_token_number': 12030, 'total_er_token_number': 5885, 'av_er_ratio': 0.48919368246051537, 'total_er_number': 3685, 'file': 'SR7C.tsv'}
{'Expression Lexicon Size without stopwords': 1514, 'file': 'SR7C.tsv'}
{'speaker1': 'S0058', 'speaker1_token_number': 3313, 'speaker1_expression_token_number': 1644, 'speaker1_expression_number': 1100, 'speaker1_er_ratio': 0.4962269846060972, 'speaker2': 'S0184', 'speaker2_token_number': 2465, 'speaker2_expression_token_number': 1217, 'speaker2

{'speaker1': 'S0104', 'speaker1_token_number': 5438, 'speaker1_expression_token_number': 2262, 'speaker1_expression_number': 1422, 'speaker1_er_ratio': 0.41596175064361895, 'speaker2': 'S0376', 'speaker2_token_number': 2652, 'speaker2_expression_token_number': 1280, 'speaker2_expression_number': 817, 'speaker2_er_ratio': 0.48265460030165913, 'speaker3': 'S0377', 'speaker3_token_number': 2379, 'speaker3_expression_token_number': 1320, 'speaker3_expression_number': 775, 'speaker3_er_ratio': 0.5548549810844893, 'total_token_number': 10469, 'total_er_token_number': 4862, 'av_er_ratio': 0.46441876014901134, 'total_er_number': 3014, 'file': 'SQPN.tsv'}
{'Expression Lexicon Size without stopwords': 1308, 'file': 'SQPN.tsv'}
{'speaker1': 'S0654', 'speaker1_token_number': 509, 'speaker1_expression_token_number': 150, 'speaker1_expression_number': 97, 'speaker1_er_ratio': 0.29469548133595286, 'speaker2': 'S0653', 'speaker2_token_number': 1201, 'speaker2_expression_token_number': 363, 'speaker2_e

{'speaker1': 'S0486', 'speaker1_token_number': 5575, 'speaker1_expression_token_number': 2034, 'speaker1_expression_number': 1258, 'speaker1_er_ratio': 0.3648430493273543, 'speaker2': 'S0523', 'speaker2_token_number': 2285, 'speaker2_expression_token_number': 1135, 'speaker2_expression_number': 686, 'speaker2_er_ratio': 0.49671772428884026, 'speaker3': 'S0489', 'speaker3_token_number': 2563, 'speaker3_expression_token_number': 1488, 'speaker3_expression_number': 844, 'speaker3_er_ratio': 0.5805696449473273, 'total_token_number': 10423, 'total_er_token_number': 4657, 'av_er_ratio': 0.44680034539000285, 'total_er_number': 2788, 'file': 'SJSQ.tsv'}
{'Expression Lexicon Size without stopwords': 1153, 'file': 'SJSQ.tsv'}
{'speaker1': 'S0422', 'speaker1_token_number': 512, 'speaker1_expression_token_number': 185, 'speaker1_expression_number': 118, 'speaker1_er_ratio': 0.361328125, 'speaker2': 'S0423', 'speaker2_token_number': 2238, 'speaker2_expression_token_number': 977, 'speaker2_expressio

{'speaker1': 'S0475', 'speaker1_token_number': 882, 'speaker1_expression_token_number': 100, 'speaker1_expression_number': 67, 'speaker1_er_ratio': 0.11337868480725624, 'speaker2': 'S0418', 'speaker2_token_number': 20, 'speaker2_expression_token_number': 6, 'speaker2_expression_number': 6, 'speaker2_er_ratio': 0.3, 'speaker3': 'S0417', 'speaker3_token_number': 782, 'speaker3_expression_token_number': 318, 'speaker3_expression_number': 235, 'speaker3_er_ratio': 0.40664961636828645, 'total_token_number': 1684, 'total_er_token_number': 424, 'av_er_ratio': 0.2517814726840855, 'total_er_number': 308, 'file': 'SKY2.tsv'}
{'Expression Lexicon Size without stopwords': 140, 'file': 'SKY2.tsv'}
{'speaker1': 'S0278', 'speaker1_token_number': 5207, 'speaker1_expression_token_number': 2028, 'speaker1_expression_number': 1325, 'speaker1_er_ratio': 0.38947570578067986, 'speaker2': 'S0013', 'speaker2_token_number': 2292, 'speaker2_expression_token_number': 1169, 'speaker2_expression_number': 728, 'spe

{'speaker1': 'S0013', 'speaker1_token_number': 391, 'speaker1_expression_token_number': 229, 'speaker1_expression_number': 145, 'speaker1_er_ratio': 0.5856777493606138, 'speaker2': 'S0152', 'speaker2_token_number': 10616, 'speaker2_expression_token_number': 2130, 'speaker2_expression_number': 1462, 'speaker2_er_ratio': 0.2006405425772419, 'speaker3': 'S0012', 'speaker3_token_number': 2426, 'speaker3_expression_token_number': 1627, 'speaker3_expression_number': 1015, 'speaker3_er_ratio': 0.6706512778235779, 'total_token_number': 13433, 'total_er_token_number': 3986, 'av_er_ratio': 0.29673192883198096, 'total_er_number': 2622, 'file': 'S6A5.tsv'}
{'Expression Lexicon Size without stopwords': 909, 'file': 'S6A5.tsv'}
{'speaker1': 'S0653', 'speaker1_token_number': 3436, 'speaker1_expression_token_number': 1307, 'speaker1_expression_number': 861, 'speaker1_er_ratio': 0.38038416763678695, 'speaker2': 'S0655', 'speaker2_token_number': 1180, 'speaker2_expression_token_number': 457, 'speaker2_e

In [45]:
#get expression lexicon
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-3/output_er_summary_3_dialign_lexicon'

import glob,os
files = glob.glob('S***.tsv')
ex_general_summary = []
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,3)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
    ex_for_each_file = []
    for ex, value in ex_dic_f.items():
        ex_for_each_file.append({'Expression':ex, 'Expression_length': len(ex.split()), 'Expression_established_position':value[0], 'Expression_uttered_position':value[1], 'file':f})
    
    ex_general_summary.append({'Expression Lexicon Size': len(ex_dic_f),'file':f})
    print({'Expression Lexicon Size': len(ex_dic_f),'file':f})
    
    dl_ex_for_each_file = pd.DataFrame(ex_for_each_file)
    dl_ex_for_each_file.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)
    
dl_ex_general_summary = pd.DataFrame(ex_general_summary)
dl_ex_general_summary.to_csv(os.path.join(path,'expression_lexicon_size_3_1.csv'), index = False)






{'Expression Lexicon Size': 318, 'file': 'SWHR.tsv'}
{'Expression Lexicon Size': 1191, 'file': 'SWSX.tsv'}
{'Expression Lexicon Size': 2865, 'file': 'SR4E.tsv'}
{'Expression Lexicon Size': 871, 'file': 'SK82.tsv'}
{'Expression Lexicon Size': 648, 'file': 'SFRW.tsv'}
{'Expression Lexicon Size': 700, 'file': 'SDQ8.tsv'}
{'Expression Lexicon Size': 854, 'file': 'SVLZ.tsv'}
{'Expression Lexicon Size': 388, 'file': 'SX4N.tsv'}
{'Expression Lexicon Size': 686, 'file': 'SV5R.tsv'}
{'Expression Lexicon Size': 270, 'file': 'SHKZ.tsv'}
{'Expression Lexicon Size': 683, 'file': 'SAAF.tsv'}
{'Expression Lexicon Size': 1448, 'file': 'SA88.tsv'}
{'Expression Lexicon Size': 776, 'file': 'S8NX.tsv'}
{'Expression Lexicon Size': 1451, 'file': 'S8LG.tsv'}
{'Expression Lexicon Size': 951, 'file': 'S49V.tsv'}
{'Expression Lexicon Size': 759, 'file': 'S4NB.tsv'}
{'Expression Lexicon Size': 1561, 'file': 'SQP2.tsv'}
{'Expression Lexicon Size': 857, 'file': 'SNQD.tsv'}
{'Expression Lexicon Size': 577, 'file': 

{'Expression Lexicon Size': 429, 'file': 'SMH4.tsv'}
{'Expression Lexicon Size': 2240, 'file': 'SLNB.tsv'}
{'Expression Lexicon Size': 569, 'file': 'SKDP.tsv'}
{'Expression Lexicon Size': 1470, 'file': 'SEJT.tsv'}
{'Expression Lexicon Size': 314, 'file': 'SK3S.tsv'}
{'Expression Lexicon Size': 662, 'file': 'S2PS.tsv'}
{'Expression Lexicon Size': 1225, 'file': 'S5U8.tsv'}
{'Expression Lexicon Size': 2202, 'file': 'S985.tsv'}
{'Expression Lexicon Size': 5092, 'file': 'S5XD.tsv'}
{'Expression Lexicon Size': 2525, 'file': 'SWWZ.tsv'}
{'Expression Lexicon Size': 921, 'file': 'SUAB.tsv'}
{'Expression Lexicon Size': 418, 'file': 'SYWB.tsv'}
{'Expression Lexicon Size': 725, 'file': 'SCG9.tsv'}
{'Expression Lexicon Size': 327, 'file': 'STDN.tsv'}
{'Expression Lexicon Size': 241, 'file': 'S4KE.tsv'}
{'Expression Lexicon Size': 699, 'file': 'SXGX.tsv'}
{'Expression Lexicon Size': 2848, 'file': 'SMRV.tsv'}
{'Expression Lexicon Size': 1252, 'file': 'SDZC.tsv'}
{'Expression Lexicon Size': 1014, 'fil

{'Expression Lexicon Size': 409, 'file': 'SKBN.tsv'}
{'Expression Lexicon Size': 520, 'file': 'SBKH.tsv'}
{'Expression Lexicon Size': 800, 'file': 'SRX4.tsv'}
{'Expression Lexicon Size': 2662, 'file': 'S7JG.tsv'}
{'Expression Lexicon Size': 199, 'file': 'SKY2.tsv'}
{'Expression Lexicon Size': 1182, 'file': 'SKYQ.tsv'}
{'Expression Lexicon Size': 376, 'file': 'S9D7.tsv'}
{'Expression Lexicon Size': 1261, 'file': 'SPTA.tsv'}
{'Expression Lexicon Size': 1085, 'file': 'S539.tsv'}
{'Expression Lexicon Size': 1436, 'file': 'SVUC.tsv'}
{'Expression Lexicon Size': 2650, 'file': 'SMME.tsv'}
{'Expression Lexicon Size': 1524, 'file': 'S66A.tsv'}
{'Expression Lexicon Size': 1284, 'file': 'SHJE.tsv'}
{'Expression Lexicon Size': 2520, 'file': 'SF2Q.tsv'}
{'Expression Lexicon Size': 528, 'file': 'S3SU.tsv'}
{'Expression Lexicon Size': 612, 'file': 'S84P.tsv'}
{'Expression Lexicon Size': 1009, 'file': 'S6A5.tsv'}
{'Expression Lexicon Size': 733, 'file': 'SXC6.tsv'}
{'Expression Lexicon Size': 3923, 'f